In [1]:
# raise ValueError
from sklearnex import patch_sklearn
patch_sklearn()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


# Modeling exercise

## General Instructions

* Submission date: 25.4.2022
* Submission Method: Link to your solution notebook in [this sheet](https://docs.google.com/spreadsheets/d/1fTmjiVxzw_rM1hdh16enwUTtxzlHSJIiw41dJS2LKp0/edit?usp=sharing).

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys; sys.path.append('../Modles and Modeling/src')
import numpy as np
import plotly_express as px

In [4]:
import pandas as pd
import ipywidgets as widgets

In [5]:
from datasets import make_circles_dataframe, make_moons_dataframe

## Fitting and Overfiting 

The goal of the following exercise is to:
* Observe overfitting due to insuffient data
* Observe Overfitting due to overly complex model
* Identify the overfitting point by looking at Train vs Test error dynamic
* Observe how noise levels effect the needed data samples and model capacity

To do so, you'll code an experiment in the first part, and analyze the experiment result in the second part.

### Building an experiment

Code:

1. Create data of size N with noise level of magnitude NL from datasets DS_NAME. 
1. Split it to training and validation data (no need for test set), use 80%-20%. 
1. Use Logistic regression and Choose one complex model of your choice: [KNN](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html), [SVM with RBF kernel](https://scikit-learn.org/stable/modules/svm.html) with different `gamma` values or [Random forest classifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) with differnt number of `min_samples_split`. 
1. Train on the train set for different hyper parameter values. compute:
   1. Classification accuracy on the training set (TRE)
   1. Classification accuracy on the validation set (TESTE)
   1. The difference beteen the two above (E_DIFF)
1. Save DS_NAME, N, NL, CLF_NAME, K, TRE, TESTE, E_DIFF and the regularization/hyper param (K, gamma or min_samples_split and regularization value for the linear regression classifier)

Repeat for:
* DS_NAME in Moons, Circles
* N (number of samples) in [5, 10, 50, 100, 1000, 10000]
* NL (noise level) in [0, 0.1, 0.2, 0.3, 0.4, 0.5]
* For the complex model: 10 Values of hyper parameter of the complex model you've chosen.
* For the linear model: 5 values of ridge (l2) regularization - [0.001, 0.01, 0.1, 1, 10, 100, 1000]

### Analysing the expermient results

1. For SVM only, For dataset of size 10k and for each dataset, What are the best model params? How stable is it? 
1. For SVM only, For dataset of size 10k and for each dataset, What is the most stable model and model params? How good is it in comparison to other models? Explain using bias and variance terminoligy.
1. Does regularization help for linear models? consider different datasets sizes. 
1. For a given noise level of your chioce, How does the train, test and difference error changes with increasing data sizes? (answer for svm and LR seperatly)
1. For a given noise level of your chioce, How does the train, test and difference error changes with increasing model complexity? (answer for svm and LR seperatly)
1. Are the noise level effect the number of datapoints needed to reach optimal test results? 

Bonus:

* For SVM: Select one dataset and with 0.2 noise level. Identify the optimal model params, and visualize the decision boundry learned. 
  * Hint: Use a grid. See classification models notebook 

## Tips and Hints

For buliding the experiment:

* Start with one dataframe holding all the data for both datastes with different noise level. Use the `make_<dataset_name>_dataframe()` functions below, and add two columns, dataset_name and noise_level, before appending the new dataset to the rest of the datasets. Use `df = pd.DataFrame()` to start with an empty dataframe and using a loop, add data to it using `df = df.append(<the needed df here>)`. Verify that you have 10k samples for each dataset type and noise level by a proper `.value_counts()`. You can modify the 
* When you'll need an N samples data with a specific noise level, use `query()` and `head(n)` to get the needed dataset. 
* Use sklearn `train_test_split()` method to split the data with `test_size` and `random_state` parameters set correctly to ensure you are always splitting the data the same why for a given fold `k`. Read [the docs](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) if needed. 
* You can also not create your own data splitter, and instead use `model_selection.cross_validate()` from sklearn. You'll need to ask for the train erros as well as the test errors, see [here](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html).
* Use prints in proper location to ensure the progress of the experiment. 

**If you get stuck, and need refernce, scroll to the end of the notebook to see more hints!**

## Moons dataset

In [6]:
from sklearn.datasets import make_moons

In [7]:
moons_df = make_moons_dataframe(n_samples=1000, noise_level=0.1)
moons_df.head()

,x,y,label
0,-0.815277,0.481594,A
1,0.593989,-0.227605,B
2,0.194304,0.108145,B
3,0.121799,0.308393,B
4,-0.517474,0.603071,A


In [8]:
@widgets.interact
def plot_noisy_moons(noise_level = widgets.FloatSlider(value=0, min=0, max=0.5, step=0.05)):
    moons_df = make_moons_dataframe(n_samples=1000, noise_level=noise_level)
    return px.scatter(moons_df, x='x', y='y', color = 'label')

interactive(children=(FloatSlider(value=0.0, description='noise_level', max=0.5, step=0.05), Output()), _dom_c…

## Circles Dataset

In [9]:
circles_df = make_circles_dataframe(n_samples=500, noise_level=0)
circles_df.head()

,x,y,label
0,-0.509939,0.616411,B
1,-0.070281,-0.796907,B
2,0.062791,0.998027,A
3,0.999684,-0.025130,A
4,-0.149905,0.785830,B


In [10]:
@widgets.interact
def plot_noisy_circles(noise_level = widgets.FloatSlider(value=0, min=0, max=0.5, step=0.05)):
    df = make_circles_dataframe(1000, noise_level)
    return px.scatter(df, x='x', y='y', color = 'label')

interactive(children=(FloatSlider(value=0.0, description='noise_level', max=0.5, step=0.05), Output()), _dom_c…

## Solution:

In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from itertools import product
from random import shuffle

In [12]:
def split_data(data):
    """
    Randomly split the input data into 2 sets: Train set, and Validation set.
    
    data : pandas DataFrame
    
    return:
    train_data : pandas DataFrame
    valid_data : pandas DataFrame
    """
    n = data.shape[0]
    data = data.copy().sample(n) # Randomized copy
    data = data.reset_index(drop=True) # "Forget" original index order, and reindex based on random order.
                              
    n_train = np.round(n*0.8)
    # n_valid = n - n_train  # ~20%
    mask = data.index < n_train                          
    
    train_data = data.loc[mask,:]
    valid_data = data.loc[~mask,:]

    return train_data, valid_data

Note: for some reason, section 4.A and 4.B ask for the "accuracy" with the misnomer TRE / TESTE.
It would make more sense to ask for the training ERROR and the test ERROR, but following this exercise instructions, the accoracies will be reported.

4A Classification accuracy on the training set (TRE)

4B Classification accuracy on the validation set (TESTE)

In [13]:
cols = ["DS_NAME", "N", "NL", "CLF_NAME", "K", "Hyperparameter", "TRE", "TESTE", "E_DIFF",  "TR_ACC", "TEST_ACC", "ACC_DIFF"]
# DS_NAME, N, NL, CLF_NAME, k, Hyperparameter, TRE, TESTE, E_DIFF, TR_ACC, TEST_ACC, ACC_DIFF

df = pd.DataFrame(columns=cols)
print("Empty summary df:")
df

Empty summary df:


,DS_NAME,N,NL,CLF_NAME,K,Hyperparameter,TRE,TESTE,E_DIFF,TR_ACC,TEST_ACC,ACC_DIFF


Models characteristics:

DS_NAME in Moons, Circles
N (number of samples) in [5, 10, 50, 100, 1000, 10000]
NL (noise level) in [0, 0.1, 0.2, 0.3, 0.4, 0.5]
For the complex model: 10 Values of hyper parameter of the complex model you've chosen.
For the linear model: 5 values of ridge (l2) regularization - [0.001, 0.01, 0.1, 1, 10, 100, 1000]

In [14]:
N = [5, 10, 50, 100, 1000, 10000]
NL = [0, 0.1, 0.2, 0.3, 0.4, 0.5]
DS_NAME = ["Moons", "Circles"]
CLF_NAME = ["LogisticRegression", "SVM"]
Hyperparameter = [0.001, 0.01, 0.1, 1, 10, 100, 1000]
k_folds = 10
K = [k for k in range(k_folds)]

models_ls = list(product(N, NL, DS_NAME, CLF_NAME, K, Hyperparameter))

In [15]:
# idx_ls = list(np.random.randint(low=0, high=len(models_ls),size=5))
# print("\nExamples for (N, NL, DS_NAME, CLF_NAME, Hyperparameter):\n")
# for idx in idx_ls:
#     print(models_ls[idx])

In [16]:
# shuffle(models_ls) # for better progress report
print("\nExamples for (N, NL, DS_NAME, CLF_NAME, Hyperparameter):\n")
# models_ls = models_ls[:5]
# models_ls = models_ls[:5]


Examples for (N, NL, DS_NAME, CLF_NAME, Hyperparameter):



In [17]:
len(models_ls)

10080

Learning process:

In [ ]:
# cols = ["DS_NAME", "N", "NL", "CLF_NAME", "Hyperparameter", "TRE", "TESTE", "E_DIFF"]
df = pd.DataFrame(columns=cols)

last_iter = None
last_save = None
# k_folds = 10
# print("Processing...")
# def calculate_model(model_params):
#     print("Processing...")
#     N, NL, DS_NAME, CLF_NAME, Hyperparameter = model_params
for iteration, (N, NL, DS_NAME, CLF_NAME, k, Hyperparameter) in enumerate(models_ls):
#     TRE, TESTE, E_DIFF = [None]*3
    # break # already saved in excel

    if iteration < 9590:
        continue
    # Create the data
    if DS_NAME=="Moons":
        data = make_moons_dataframe(n_samples=N, noise_level=NL)
    elif DS_NAME=="Circles":
        data = make_circles_dataframe(n_samples=N, noise_level=NL)
    # else:
    #     data = create_data(N=N, NL=NL, DS_NAME=DS_NAME)

    # print("Sample of",DS_NAME.__repr__,":")
    # display(data.head())

    # Split the data:
    train_data, valid_data = split_data(data)

    if CLF_NAME=="LogisticRegression":
        # Apply Logistic Regression:
        log_reg = LogisticRegression(penalty='l2', C=Hyperparameter)
        log_reg.fit(train_data[['x','y']], train_data['label'])

        # Retrieve the model parameters.
        # b = log_reg.intercept_[0]
        # w1, w2 = log_reg.coef_[0]

        # Calculate the intercept and gradient of the decision boundary.
        # c = -b/w2
        # m = -w1/w2

        # Predict train_data:
        train_data['predicted_label'] = log_reg.predict(train_data[['x','y']])
        # train_data[['prob_A','prob_B']] = log_reg.predict_proba(train_data[['x','y']])
        # train_data['log_regression_value'] = train_data.x.apply(lambda x: m*x + c)

        # Predict valid_data:
        valid_data['predicted_label'] = log_reg.predict(valid_data[['x','y']])
        # valid_data[['prob_A','prob_B']] = log_reg.predict_proba(valid_data[['x','y']])
        # valid_data['log_regression_value'] = valid_data.x.apply(lambda x: m*x + c)

    elif CLF_NAME=="SVM":
        svm_clf = svm.SVC(kernel='poly', degree=2, gamma=Hyperparameter) # kernels availble: 'linear', 'poly', 'rbf'
        svm_clf.fit(train_data[['x','y']], train_data['label'])
        # continue implementation of SVM
        # Predict train_data:
        train_data['predicted_label'] = svm_clf.predict(train_data[['x','y']])
        # train_data[['prob_A','prob_B']] = svm_clf.predict_proba(train_data[['x','y']])
        # train_data['log_regression_value'] = train_data.x.apply(lambda x: m*x + c)

        # Predict valid_data:
        valid_data['predicted_label'] = svm_clf.predict(valid_data[['x','y']])
        # valid_data[['prob_A','prob_B']] = svm_clf.predict_proba(valid_data[['x','y']])
        # valid_data['log_regression_value'] = valid_data.x.apply(lambda x: m*x + c)


    # Calculate performance:
    train_data["correct_prediction"] = train_data.label == train_data.predicted_label
    TR_ACC = np.mean(train_data.loc[:,"correct_prediction"]) # TRain set accuracy
    TRE = 0 # len(train_data.loc[:,"correct_prediction"]) - np.sum(train_data.loc[:,"correct_prediction"]) # TRain set Error
    
    valid_data["correct_prediction"] = valid_data.label == valid_data.predicted_label
    TEST_ACC = np.mean(valid_data.loc[:,"correct_prediction"]) # Validation (test) set accuracy
    TESTE = 0 #len(valid_data.loc[:,"correct_prediction"]) - np.sum(valid_data.loc[:,"correct_prediction"]) # Validation (test) set Error
    
    ACC_DIFF = 0 # np.abs(TR_ACC - TEST_ACC)
    E_DIFF = 0 #np.abs(TESTE - TRE)

    # Report:
    # print(f"\nResults of {CLF_NAME} on {DS_NAME}:")
    # print(f"E_DIFF = {E_DIFF}  \nTESTE = {TESTE} \nTRE = {TRE}")
    # perf_dic = {DS_NAME:"DS_NAME", N:"N", NL:"NL", CLF_NAME:"CLF_NAME", Hyperparameter:"Hyperparameter", TRE:"TRE", TESTE:"TESTE", E_DIFF:"E_DIFF"}
    # for val in perf_dic:
        # df.loc[iteration, perf_dic[val]] = val
    ser = pd.Series([DS_NAME, N, NL, CLF_NAME, k, Hyperparameter, TRE, TESTE, E_DIFF, TR_ACC, TEST_ACC, ACC_DIFF],index=cols)
    df.loc[df.shape[0],:] = ser

    # print("\nUpdated df:")
    # display(df)

    if last_iter == None or iteration > 15+last_iter:
        last_iter = iteration
        current = round(100*iteration/len(models_ls),2)
        print(f"Current progress: {current}%")
        df.to_excel("summary_df_"+str(round(100*iteration/len(models_ls)))+".xlsx")
        # if last_save==None and iteration > len(models_ls)/5:
        #     last_save = iteration
        #     df.to_excel("summary_df_"+str(round(100*iteration/len(models_ls)))+".xlsx")
        # elif last_save is not None and iteration > last_save+len(models_ls)/20:
        #     df.to_excel("summary_df_"+str(round(100*iteration/len(models_ls)))+".xlsx")

    # return None
display(df)
df.to_excel("summary_df.xlsx")

Current progress: 95.14%
Current progress: 95.3%
Current progress: 95.46%


In [ ]:
raise ValueError

# df.to_excel("summary_df.xlsx")

# Load pre-saved df from Excel:
# df = pd.read_excel("summary_df_91_8pm_2404.xlsx", index_col=0) # Unfortunately, only 91% of the models were saved (PC broke before it finished)
df_res = pd.read_csv(r"C:\Users\hagai\Desktop\Uriel\cv_df.csv")
df_res = df_res.replace({"circles":"Circles", "moons":"Moons", "log_reg":"LogisticRegression", "svm":"SVM"})
df_res = df_res.rename(columns={"HP":"Hyperparameter"})
display(df_res)

In [ ]:
# df[:50]

In [ ]:
df_mean = df_res.groupby(["DS_NAME", "N","CLF_NAME", "NL", "Hyperparameter"])[["TRE", "TESTE", "E_DIFF"]].mean()
df_mean = df_mean.rename(columns=lambda x: x+"_mean")
df_std = df_res.groupby(["DS_NAME", "N", "CLF_NAME", "NL", "Hyperparameter"])[["TRE", "TESTE", "E_DIFF"]].std()
df_std = df_std.rename(columns=lambda x: x+"_std")
df_std
df = pd.concat([df_mean, df_std], axis=1)
df.tail(60)

In [ ]:
# 1. For SVM only, For dataset of size 10k and for each dataset, What are the best model params? How stable is it?
moons_q1 = df.loc[("Moons","10000","SVM"),:]
moons_q1 = moons_q1.reset_index()
moons_q1.head(5)

In [ ]:
fig_m1 = px.line(x=moons_q1.Hyperparameter, y=[moons_q1.TRE_mean, moons_q1.TESTE_mean], facet_col=moons_q1.NL, log_x=True,
              labels={"x":"gamma", "value":"Accuracy", "facet_col":"Noise level", "variable":"Accuracy", "wide_variable_0":"TRE", "wide_variable_1":"TESTE"},
              title="Moons at n=10k<br>Accuracy for gamma (log axis) in each noise level")
fig_m1.update_layout(title_x=0.5)
fig_m1.show()

A to Q1:

In [ ]:
# 1. For SVM only, For dataset of size 10k and for each dataset, What are the best model params? How stable is it?

moons_q1 = df[(df.CLF_NAME=="SVM") & (df.N==10000) & (df.DS_NAME=="Moons")][["NL", "K", "Hyperparameter", "TRE", "TESTE", "E_DIFF"]]
display(moons_q1.head())


fig_m1 = px.line(x=moons_q1.Hyperparameter, y=[moons_q1.TRE, moons_q1.TESTE], facet_col=moons_q1.NL, log_x=True, facet_row= moons_q1.K,
              labels={"x":"gamma", "value":"Accuracy", "facet_col":"Noise level", "variable":"Accuracy", "wide_variable_0":"TRE", "wide_variable_1":"TESTE"},
              title="Moons at n=10k<br>Accuracy for gamma (log axis) in each noise level")
fig_m1.update_layout(title_x=0.5)
fig_m1.show()

circles_q1 = df[(df.CLF_NAME=="SVM") & (df.N==10000) & (df.DS_NAME=="Circles")][["NL", "K", "Hyperparameter", "TRE", "TESTE", "E_DIFF"]]
display(circles_q1.head())


fig_c1 = px.line(x=circles_q1.Hyperparameter, y=[circles_q1.TRE, circles_q1.TESTE], facet_col=circles_q1.NL, log_x=True, facet_row= circles_q1.K,
              labels={"x":"gamma", "value":"Accuracy", "facet_col":"Noise level", "variable":"Accuracy", "wide_variable_0":"TRE", "wide_variable_1":"TESTE"},
              title="Circles at n=10k<br>Accuracy for gamma (log axis) in each noise level")
fig_c1.update_layout(title_x=0.5)
fig_c1.show()

A to Q1:
Based on the data presented, in most cases, gamma around 1 (or above) gives the optimal results.
It is hard to speak of stability without k-fold validation which yields the standard deviation of the performance measures.

A to Q2

1. For SVM only, For dataset of size 10k and for each dataset, What are the best model params? How stable is it? 
1. For SVM only, For dataset of size 10k and for each dataset, What is the most stable model and model params? How good is it in comparison to other models? Explain using bias and variance terminoligy.
1. Does regularization help for linear models? consider different datasets sizes. 
1. For a given noise level of your chioce, How does the train, test and difference error changes with increasing data sizes? (answer for svm and LR seperatly)
1. For a given noise level of your chioce, How does the train, test and difference error changes with increasing model complexity? (answer for svm and LR seperatly)
1. Are the noise level effect the number of datapoints needed to reach optimal test results? 

## Appendix

### More hints!

If you'll build the datasets dataframe correctly, you'll have **one** dataframe that has dataset_name and noise_level colmuns, as well as the regular x,y,label colmns. To unsure you've appended everything correctly, groupby the proper colmuns and look at the size:

In [ ]:
# Use proper groupby statement to ensure the datasets dataframe contains data as expected. You should see the following result:

Your 

You experiment code should look something like that:

In [ ]:
# datasets_type = ['circles', 'moons']
# k_folds = 10
# n_samples = [10, 50, 100, 1000, 10000]
# noise_levels = [0, 0.1, 0.2, 0.3, 0.4, 0.5]
# clf_types = ['log_reg', 'svm']
# hp_range = <'Your hyper parameters ranges here'>
# regularization_values = <'Your regularization values here'>
# results = []
# for ds_type in datasets_type:
#     print(f'Working on {ds_type}')
#     for nl in noise_levels:
#         for n in n_samples:
#             ds = datasets.query(<'your query here'>).head(n)
#             print(f'Starting {k_folds}-fold cross validation for {ds_type} datasets with {n} samples and noise level {nl}. Going to train {clf_types} classifiers.')
#             for k in range(k_folds):
#                 X, Y = <'Your code here'>
#                 x_train,x_test,y_train,y_test= <'Your code here'>
#                 for clf_type in clf_types:
#                     if clf_type == 'log_reg':
#                         for regularization_value in regularization_values:
#                             train_acc, test_acc = <'Your code here'>
#                             results.append(<'Your code here'>)
#                     if clf_type == 'svm':
#                         for gamma in hp_range:
#                             train_acc, test_acc = <'Your code here'>
#                             results.append(<'Your code here'>)

### Question 1 - Manual Classification

The purpose of this excercise is to examplify the need in a fitting algorithm. We will do so by trying to find only 2 models parameters by ourselves. 

In [ ]:
slope, intercept = 2.5, 6

In [ ]:
x_1, x_2 = 0.2, 0.6
on_line = [[x, x*slope + intercept,'on_line'] for x in np.linspace(-1,2,100)]

above_line = [[x_1, x_1*slope + intercept + 2, 'Above'], 
              [x_2, x_2*slope + intercept + 2, 'Above']] 

below_line = [[x_1, x_1*slope + intercept - 2, 'Below'], 
              [x_2, x_2*slope + intercept - 2, 'Below']] 

In [ ]:
columns = ['x','y','label']
data = pd.DataFrame(on_line + above_line + below_line, columns = columns)

In [ ]:
px.scatter(data, x='x', y='y', color = 'label')